In [1]:
import pandas as pd 
import numpy as np 
import pymssql
import json 
import pandas.io.sql as psql

#SQL reading function
def mssql_to_pandas_df(sql):
    '''
    This function takes in an MSSQL query from the DWH and return as a pandas data frame.
    sql = string query in MSSQL
    Required: import pandas.io.sql as psql, import pymssql, MaelysConfig.json file to be located in the same folder

    '''

    config = json.load(open('MaelysConfig.json'))
    # Connection variables
    driver = config['driver']
    server = config['server']
    database = config['database']
    user = config['user']
    password = config['password']

    # Connection string
    cnxn = pymssql.connect(server=f'{server}.{driver}',
                            user=f'{user}@{server}', 
                            password=password, 
                            database=database) 

    # Fetch data into a cursor
    cursor = cnxn.cursor()
    df = psql.read_sql(sql, cnxn)
    return df

In [5]:
# Define range of dates
start_date = '2021-01-01'
end_date = str(pd.Timestamp.now())[:10]

sql = f'''
SELECT DISTINCT
vfs.OrderID
,vfs.BuyerEmail
,vfs.UserID
,vfs.IsLoyaltyUser
,vds.SKUNAme
,vdi.Category
,vfs.AddToCartType
,vfs.PaymentDate
,vfs.PaymentTime
,vfs.ShipmentUSD
,vfs.NetSalesUSD
,fot.DaysSinceOrder
,fot.CheckpointStatus
,vdc.CouponType
,vdc.Discount
,vfs.Country
,vfs.BuyerState
,vfs.BuyerZip
,vfs.Market
,vfs.NumberOfOrders
FROM aas.v_FactSales vfs
--------DimSKU-------
JOIN (SELECT 
            DISTINCT SKU, SKUNAme, Active, IsFree 
            FROM aas.v_DimSKU
            WHERE 1=1
            AND Active=1
            AND IsFree=0) vds 
            ON vds.SKU=vfs.SKU
--------DimItems-------
JOIN (SELECT 
            DISTINCT Category, SKU 
            FROM aas.v_DimItems) vdi 
            ON vdi.SKU=vfs.SKU
--------DimUsers--------
JOIN (SELECT 
            DISTINCT IsB2B, Gender, Email 
            FROM aas.v_DimUsers
            WHERE 1=1
            AND IsB2B<>'B2B'
            AND Gender<>'Male') vdu 
            ON vdu.Email=vfs.BuyerEmail
--------DimCoupons--------
LEFT JOIN (SELECT 
            DISTINCT Discount, CouponID, CouponType
            FROM aas.v_DimCoupons) vdc ON vdc.CouponID=vfs.CouponID
--------DeltiveyTime--------
LEFT JOIN (SELECT 
            DISTINCT OrderID, DaysSinceOrder, CheckpointStatus, RN
            FROM dwh.FactOrdersTracking
            WHERE 1=1) fot ON fot.OrderID=vfs.OrderID
WHERE 1=1
--------RetentionUsers--------
AND vfs.BuyerEmail IN (SELECT
                    DISTINCT 
                    vfs.BuyerEmail
                    FROM dwh.FactSales vfs
                    WHERE 1=1
                    AND vfs.NumberOfOrders=1
                    AND vfs.paymentdate >= '{start_date}'
                    AND vfs.paymentdate < '{end_date}')
AND vfs.NumberOfOrders IS NOT NULL
ORDER BY vfs.PaymentDate, vfs.PaymentTime
'''


In [6]:
# import data
df_original = mssql_to_pandas_df(sql)

In [7]:
# Backup data frame
df = df_original.copy()

In [8]:
df.head()

,OrderID,BuyerEmail,UserID,IsLoyaltyUser,SKUNAme,Category,AddToCartType,PaymentDate,PaymentTime,ShipmentUSD,NetSalesUSD,DaysSinceOrder,CheckpointStatus,CouponType,Discount,Country,BuyerState,BuyerZip,Market,NumberOfOrders
0,9785471,lilliensimourian@gmail.com,131482,0,1 B-TIGHT,Reshapers,Simple,2019-01-05,00:40:00,NaN,49.0,NaN,None,None,NaN,United States of America,Connecticut,06830,ROW,1
1,9785471,lilliensimourian@gmail.com,131482,0,GAP,Gap,Gap,2019-01-05,00:40:00,0.0,5.0,NaN,None,None,NaN,United States of America,Connecticut,06830,ROW,1
2,9787689,katiej1523@gmail.com,127185,0,1 B-TIGHT,Reshapers,Simple,2019-01-05,08:53:00,NaN,49.0,NaN,None,None,NaN,United States of America,Colorado,80537,ROW,2
3,9787689,katiej1523@gmail.com,127185,0,1 SILK,Face,Popup,2019-01-05,08:53:00,NaN,20.0,NaN,None,None,NaN,United States of America,Colorado,80537,ROW,2
4,9787689,katiej1523@gmail.com,127185,0,GAP,Gap,Gap,2019-01-05,08:53:00,0.0,5.0,NaN,None,None,NaN,United States of America,Colorado,80537,ROW,2


In [40]:
total_orders = len(df[(df.PaymentDate < (df.PaymentDate.max() - pd.Timedelta('45D')))].groupby('OrderID')['BuyerEmail'].count())
missing_tracking = len(df[df.DaysSinceOrder.isnull() & (df.PaymentDate < (df.PaymentDate.max() - pd.Timedelta('45D')))].groupby('OrderID')['BuyerEmail'].count())
missing_delivered = total_orders - len(df.loc[(df.CheckpointStatus=='delivered')&(df.PaymentDate < (df.PaymentDate.max() - pd.Timedelta('45D'))),'OrderID'].drop_duplicates())
undelivered = len(df.loc[(df.CheckpointStatus=='undelivered')&(df.PaymentDate < (df.PaymentDate.max() - pd.Timedelta('45D'))),'OrderID'].drop_duplicates())

pmt = missing_tracking/total_orders*100
pmd = missing_delivered/total_orders*100
umd = undelivered/total_orders*100
mdmdt = undelivered/missing_delivered*100
print(f'{round(pmt,2)}% of paid US orders in FactSales since 2021-01 do not appear in FactOrdersTracking at all.')
print(f'{round(pmd,2)}% of paid US orders in FactSales since 2021-01 do not have a row CheckPointStatus with the Value "delivered" in FactOrdersTracking.')
print(f'{round(umd,2)}% of paid US orders in FactSales since 2021-01 have CheckPointStatus with the value "undelivered" FactOrdersTracking.')
print(f'Of the {round(pmd,2)} % orders that do not have a row CheckPointStatus with the Value "delivered" in FactOrdersTracking, {round(mdmdt,2)}% have a row with the value "undelivered".')

4.93% of paid US orders in FactSales since 2021-01 do not appear in FactOrdersTracking at all.
30.61% of paid US orders in FactSales since 2021-01 do not have a row CheckPointStatus with the Value "delivered" in FactOrdersTracking.
4.62% of paid US orders in FactSales since 2021-01 have CheckPointStatus with the value "undelivered" FactOrdersTracking.
Of the 30.61 % orders that do not have a row CheckPointStatus with the Value "delivered" in FactOrdersTracking, 15.09% have a row with the value "undelivered".


In [18]:
df.loc[df.CheckpointStatus=='undelivered','OrderID'].drop_duplicates()

1210        30494612
1276        30494764
1341        30298471
1357        30446750
1403        30420525
              ...   
11817548    54894764
11830556    54966424
11839060    54984581
11841867    55026847
11861071    55114640
Name: OrderID, Length: 46431, dtype: int64

In [43]:
missing_tracking = list(df[df.DaysSinceOrder.isnull() & (df.PaymentDate < (df.PaymentDate.max() - pd.Timedelta('45D')))].groupby('OrderID')['BuyerEmail'].count().reset_index()['OrderID'])
undelivered = list(df.loc[df.CheckpointStatus=='undelivered','OrderID'].drop_duplicates().reset_index()['OrderID'])
missing_status = list(df.loc[(~df.CheckpointStatus.isin(['delivered', 'undelivered'])) & (~df.CheckpointStatus.isnull()) ,'OrderID'].drop_duplicates().reset_index()['OrderID'])

In [51]:
d = dict(MissTracking = missing_tracking, Undelivered = undelivered, MissingDeliveredOrUndelivedStatus = missing_status)
data = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items()]))